## Liquid water path (LWP) from different sources

Analysis of LWP derived from 89 GHz airborne measurements with the Microwave Radar and radiometer ([MiRAC; Mech et al. 2019]("https://amt.copernicus.org/articles/12/5019/2019/")) onboard Polar 5, ERA5 reanalysis, ICON models simulations, and the satellite products from MODIS and AMSR2 of the the 7th September 2020, a day of the MOSAiC-ACA airborne campaign.

**QUESTION**: 
  * How much LWP did we have? 
  * What is the variability of LWP along the flightrack? 
  * How does the airborne LWP compare to satellite (MODIS, AMSR), reanalysis and model (ERA5, ICON) data?

**Literature**

Some suggestions to start with are:

  * Mech, M., L.-L. Kliesch, A. Anhäuser, T. Rose, P. Kollias and S. Crewell, 2019: Microwave Radar/radiometer for Arctic Clouds MiRAC: first insights from the ACLOUD campaign, Atmospheric Measurement Techniques, 12, 5019–5037, [https://doi.org/10.5194/amt-12-5019-2019]("https://amt.copernicus.org/articles/12/5019/2019/")
  * Wendisch, M., et al., 2019: The Arctic Cloud Puzzle: Using ACLOUD/PASCAL Multi-Platform Observations to Unravel the Role of Clouds and Aerosol Particles in Arctic Amplification, Bulletin of the American Meteorological Society, 100 (5), 841-871, [https://doi.org/10.1175/BAMS-D-18-0072.1]("https://journals.ametsoc.org/view/journals/bams/100/5/bams-d-18-0072.1.xml")
  * Ruiz-Donoso, E., Ehrlich, A., Schäfer, M., Jäkel, E., Schemann, V., Crewell, S., Mech, M., Kulla, B. S., Kliesch, L.-L., Neuber, R., and Wendisch, M., 2020: Small-scale structure of thermodynamic phase in Arctic mixed-phase clouds observed by airborne remote sensing during a cold air outbreak and a warm air advection event, Atmospheric Chemistry and Physics, 20, 5487–5511, [https://doi.org/10.5194/acp-20-5487-2020]("https://acp.copernicus.org/articles/20/5487/2020/").
  * Elsaesser, G. S., O’Dell, C. W., Lebsock, M. D., Bennartz, R., Greenwald, T. J., and Wentz, F. J., 2017: The Multisensor Advanced Climatology ofLiquid Water Path (MAC-LWP), Journal of Climate, 30, 10 193–10 210, [https://doi.org/10.1175/JCLI-D-16-0902.1]("https://doi.org/10.1175/JCLI-D-16-0902.1").
  * Jacob, M., Ament, F., Gutleben, M., Konow, H., Mech, M., Wirth, M., and Crewell, S., 2019: Investigating the liquid water path over the tropicalAtlantic with synergistic airborne measurements, Atmospheric Measurement Techniques, 12, 3237–3254, [https://doi.org/10.5194/amt-12-6103237-2019]("https://doi.org/10.5194/amt-12-6103237-2019").

### Overview of research flight RF07 on 07.09.2020
In August/September 2020, the MOSAiC-ACA campaign took place in the Fam strait and around Svalbard with base in Longyearbyen. The [flight reports]("https://home.uni-leipzig.de/~ehrlich/MOSAiC_ACA_wiki_doku/lib/exe/fetch.php?media=07_20200907_mosaic-aca_flight_report.pdf") give a more detauled overview on the performed flights and conditions during the campaign. A more technical overview is on the [wiki page]("https://home.uni-leipzig.de/~ehrlich/MOSAiC_ACA_wiki_doku/doku.php").

Focus of the RF07 on 07.09.2020, the flight which is discussed in the project, was the remote sensing of the different cloud regimes North West of Svalbard.

In [1]:
%pylab inline
import matplotlib.image as mpimg
img = mpimg.imread('map_MOSAiC-ACA_20200907.png')
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
imgplot = ax.imshow(img)

Populating the interactive namespace from numpy and matplotlib


FileNotFoundError: [Errno 2] No such file or directory: 'map_MOSAiC-ACA_20200907.png'

## Liquid water path retrieval from brightness temperature observations

In general, liquid water clouds increase the brightness temperature ($T_b$) observed by microwave instruments by emission. This increase in $T_b$ makes them visible in front of the colder background, like the ocean surface with the much lower emissivity and enables us to derive the liquid water path ($LWP$) from observations. The methods to perform such a derivations are called retrieval algorithms.

Very simple but handy algorithms are linear or multiple regressions. Thereby, coefficients for fitting curves are derived based on a sufficiently large amount of pairs of observed $T_bs$ and $LWP$s. Such pairs can be generated from atmospheric model runs in combination with radiative transfer models. Here we use the ICOsahedral Non-hydrostatic (ICON) model together with the Passive and Active Microwave TRAnsfer model ([PAMTRA; Mech et al., 2020]("https://gmd.copernicus.org/articles/13/4229/2020/")) to generate a database of $T_b$s and $LWP$s. In the algortihm used here, the difference between cloudy and nearby clear sky $T_b$ is used as input for the retrieval. Since observations are difficult over sea ice or the marginal sea ice zone (MIZ) due to their unknown surface emissivity (the one over ocean can be calculated), only sea ice free observations and simulations are used. 

In this example we use a cubic regression for the derivation of the liquid water path.

$LWP = c_1 (\Delta T_b) + c_2 (\Delta T_b)^2 + c_3 (\Delta T_b)^3 ~~~~ \mathrm{with} ~~~~ \Delta T_b = T_b - T_0$

The following steps describe how the coefficients are generated.

### Collcection of methods for the project work

In [ ]:
from scipy.spatial import KDTree

class field_kdtree(object):
    """
    KDtree are described on for example on https://towardsdatascience.com/using-scikit-learns-binary-trees-to-efficiently-find-latitude-and-longitude-neighbors-909979bd929b

    """

    def __init__(self, field_lat,field_lon,isRad=True):
        
        self.kdtree = {}
        
        if isRad:
            field_lat = np.rad2deg(field_lat)
            field_lon = np.rad2deg(field_lon)
        self.kdtree = KDTree(np.asarray([field_lat.flatten(),field_lon.flatten()]).T)

    def query(self,path_ds):

        distance, indices1d = self.kdtree.query(np.asarray([
                    path_ds.lat.values,
                    path_ds.lon.values
                ]).T)


        return indices1d

In [ ]:
import gdal
import cartopy.crs as ccrs
import pandas as pd

In [ ]:
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    return ds
    
# Opens the data HDF file and returns as a dataframe
def read_dataset(SUBDATASET_NAME, FILEPATH):
    dataset = load_data(FILEPATH)
    path = ''
    for sub, description in dataset.GetSubDatasets():
        if (description.endswith(SUBDATASET_NAME)):
            path = sub
            break
    if(path == ''):
        print(SUBDATASET_NAME + ' not found')
        return
    subdataset = gdal.Open(path)
    subdataset = subdataset.ReadAsArray()
    subdataset = pd.DataFrame(subdataset)
    return subdataset

### Open PAMTRA simulations

PAMTRA simulations for 89 GHz based on ICON atmospheric model simulations for the 07.09.2020. Only profiles over ocean have been included. lat/lon and the water paths for cloud, ice, rain, snow, graupel, and hail are directly the integrated values from the ICON model relevant for the retrieval. 

In [ ]:
import xarray as xr

In [ ]:
ds_pam = xr.open_dataset('data/pamtra_icon_passive.nc') # open data set with PAMTRA simulations

The MiRAC radar is mounted with 25° backwards. This is to reduce the strong surface reflectivity which would lead to saturation in the receiver. In the simulations we have to consider the same angle.

In [ ]:
tb = ds_pam['tb'][:,0,0,4,0,1] # dims(nx,ny,nf,nang,nobs,npol) get the horizontal (npol=1) polarization at ~25° (nang=4), cause MiRAC-A 89 GHz is horizontally polarized

In [ ]:
twp = ds_pam['cwp']+ds_pam['iwp']+ds_pam['rwp']+ds_pam['swp']+ds_pam['gwp']+ds_pam['hwp'] # this is the total integrated hydrometeor content from the ICON simulation

In [ ]:
plot(tb[twp[:,0] < 0.00001],'r.') # brightness temperature where total water path (twp) content is very small

In [ ]:
tb_clear = tb[twp[:,0] < 0.00001].mean() #  find the mean clear sky brightness temperature

In [ ]:
tb_diff = tb - tb_clear # this is the field of brightness temperature differences that enters the retrieval development

### Develop retrieval coefficients for liquid water path

The process of finding the coefficients of a regression or a neural network is called developping. Here use the *curve_fit* method from *scipy.optimize*. 

In [ ]:
from scipy.optimize import curve_fit

Define a function that does the regression. Here we use a cubic function.

In [ ]:
def cubic_regression(x,c1, c2, c3):return c1*x+c2*x**2+c3*x**3

Get parameters for the cubic regression that will be applied to the brightness temperature differences.

In [ ]:
c1,c2,c3 = curve_fit(cubic_regression, tb_diff.values,ds_pam['cwp'][:,0].values)[0]

Apply coefficients to simulated brightness temperatures as self test

In [ ]:
lwp = c1*tb_diff+c2*tb_diff**2+c3*tb_diff**3

In [ ]:
scatter(lwp,ds_pam['cwp'][:,0],c=twp)
plot([0,1,2],'k')
xlabel('LWP retrieved [$\mathrm{kg/{m^2}}$]')
ylabel('LWP simulated [$\mathrm{kg/{m^2}}$]')

### Read in  airborne data

For the comparison of liquid water path between airborne observations, model and reanalysis output, and satellite measurements we concentrate on observations over ocean from the 07.09.2020, RF07 of the [MOSAiC-ACA campaign]("https://home.uni-leipzig.de/~ehrlich/MOSAiC_ACA_wiki_doku/doku.php?id=start")

In [ ]:
import ac3airborne # load the ac3 airborne module

In [ ]:
cat = ac3airborne.get_intake_catalog() # get the intake catalogue for the data

In [ ]:
flight_id = 'MOSAiC-ACA_P5_RF07' # define flight

In [ ]:
seaice = cat.Polar5.AMSR2_SIC[flight_id].to_dask() #  get sea ice data

In [ ]:
def consecutive_valid(data, N):
    """
    This function selects all sections longer than N consecutive data points that are not equal nan.
    """
    #if (N % 2) != 0: N=2*(N//2)+1 # check for add N
    #return data.loc[np.isfinite(data.rolling(time=N, min_periods=N, center=True).sum().rolling(time=N, min_periods=1, center=True).sum())]
    return data.loc[np.isfinite(data[::-1].rolling(time=N, min_periods=N, center=True).sum()[::-1].rolling(time=N, min_periods=1, center=True).sum())]


In [ ]:
open_ocean = seaice.where(seaice.sic == 0)

In [ ]:
long_ocean_sections = consecutive_valid(open_ocean['sic'],60*30).dropna('time')

In [ ]:
track = cat.Polar5.GPS_INS[flight_id].to_dask() # get position and attitude of platform

In [ ]:
track_ocean = track.loc[dict(time=long_ocean_sections.time)]

#### Cut out corners

In [ ]:
meta = ac3airborne.get_flight_segments() # Get flight segments

In [ ]:
import pandas as pd
time_rng = pd.date_range(datetime.datetime.now().date(), periods=0, freq='S') # define empty pandas time range

Get all high level segments of research flight _MOSAiC-ACA_P5_RF07_  and stick them together.

In [ ]:
for seg in meta['P5'][flight_id]['segments']:
    if 'high_level' in seg['kinds']:
        time_rng = time_rng.union(pd.date_range(seg['start'],seg['end'],freq='S'))

In [ ]:
track_ocean_straight_legs = track_ocean.loc[dict(time=isin(track_ocean.time,time_rng))]

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
data_crs = ccrs.PlateCarree()
map_crs = ccrs.NorthPolarStereo()
fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw=dict(projection=map_crs))
ax.coastlines()
kwargs = dict(s=4, color='k', linewidths=0, transform=data_crs)
ax.scatter(track.lon, track.lat, **kwargs)
kwargs = dict(s=4, color='r', linewidths=0, transform=data_crs)
ax.scatter(track_ocean.lon, track_ocean.lat, **kwargs)
kwargs = dict(s=4, color='g', linewidths=0, transform=data_crs)
ax.scatter(track_ocean_straight_legs.lon, track_ocean_straight_legs.lat, **kwargs)

Clean up dataset before storing.

In [ ]:
track_ocean_straight_legs.attrs = dict()

In [ ]:
track_ocean_straight_legs.drop_vars(['tas','vs','gs','pitch','roll','alt','heading']).to_netcdf('polar5_ocean_track_straight_legs.nc')

_polar5_ocean_track_straight_legs.nc_ contains all times, latitudes, and longitudes that belong to observations along a straight line over open ocean. 

#### Get airborne brightness temperature measurements

### Datasets and ToDos

  * MiRAC airborne observations: _mirac_tbs.nc_
  * PAMTRA simulations including ICON field: _pamtra_icon_passive.nc_
  * ERA5 liquied water path field: _era5-gl_20200907_lwp.nc_
  * MODIS observations and geolocation: _MOD06_L2.A2020251.1055.061.2020251195046.hdf_, _MOD03.A2020251.1055.061.2020251164154.hdf_
  * AMSR2 dataset: _GW1AM2_202009070848_026A_L2SGCLWLD2210210.h5_

Except for MODIS, all datasets are readable as xarrays. For MODIS, the routines are provided.

The python class *field_kdtree* can be used to cut a path from a model field. In the initialization the field is passed and the query methods results in the indices of the field that are next to the path positions. 